# Setting up the Fast Withdrawal contract with a proxy:
- This notebook allows deploying contracts for fast withdrawal interactions and running a basic scenario by creating a withdrawal on the Etherlink side and making fast payouts on the Tezos side.

In [1]:
from docs.scenarios.setup import *

web3, etherlink_account, tezos_account = setup()

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `237.821049 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `610.537924879 ꜩ`


## Deploy FastWithdrawal:

In [2]:
from scripts.tezos import deploy_fast_withdrawal

fast_withdrawal = deploy_fast_withdrawal.callback(
    exchanger_address=EXCHANGER_ADDRESS,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    skip_confirm=True,
    silent=False,
)

Deploying Fast Withdrawal contract:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/ghostnet/`
  - Params:
      * Exchanger address: `KT1Bp9YUvUBJgXxf5UrYTM2CGRUPixURqx4m`
      * Smart Rollup address: `sr18wx6ezkeRjt1SZSeZ2UQzQN3Uc3YLMLqg`
Successfully deployed Fast Withdrawal, address: `KT18zhkJ9AohD17HaPuEiMfvjQC9horkn6Xb`


## Deploy PurchaseWithdrawalProxy:

In [3]:
from scripts.tezos import deploy_purchase_withdrawal_proxy

purchase_withdrawal_proxy = deploy_purchase_withdrawal_proxy.callback(
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    skip_confirm=True,
    silent=False,
)

Deploying Purchase Withdrawal Proxy contract:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/ghostnet/`
Successfully deployed Purchase Withrawal Proxy, address: `KT1Log35YrDTHBski4ZH7tJnzHedz5B2crb5`


## Making Fast Withdrawal on Etherlink side:

In [8]:
from scripts.etherlink import xtz_fast_withdraw

xtz_fast_withdraw.callback(
    target=get_address(tezos_account),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=414 * 10**12,
    discounted_amount=400,
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Fast Withdrawal contract: `KT18zhkJ9AohD17HaPuEiMfvjQC9horkn6Xb`
      * Payload bytes: `05009006`
      * Amount (mutez): `414`
      * Discounted amt (mutez): `400`
      * Fee (mutez): `14`
Successfully initiated FA withdrawal, tx hash: `0xee1603d24fa6b913afaff236ab398706ae07ce3b894dfd92b53a47c95252db2c`


'0xee1603d24fa6b913afaff236ab398706ae07ce3b894dfd92b53a47c95252db2c'

- Event logs: https://testnet.explorer.etherlink.com/tx/0xee1603d24fa6b913afaff236ab398706ae07ce3b894dfd92b53a47c95252db2c?tab=logs
- Outbox message: https://ghostnet-smart.tzkt.io/global/block/11218870/outbox/11218870/messages
- Withdrawal ID: `1811`
- Timestamp: `1742129936`
- [ ] TODO: Retrieve `withdrawal_id` and `timestamp` from the Etherlink side event/outbox; currently, it is set manually.

## Purchasing withdrawal on Tezos side:

In [15]:
from scripts.helpers.contracts.fast_withdrawal import Withdrawal
from scripts.helpers.utility import pack

discounted_amount = 400

withdrawal = Withdrawal(
    withdrawal_id=1811,
    full_amount=414,
    timestamp=1742129936,
    base_withdrawer=get_address(tezos_account),
    payload=pack(discounted_amount, 'nat'),
    l2_caller=bytes.fromhex(etherlink_account.address.split('0x')[1])
)

opg = purchase_withdrawal_proxy.purchase_withdrawal_proxy(
    withdrawal=withdrawal,
    service_provider=get_address(tezos_account),
    fast_withdrawal_contract=fast_withdrawal,
    exchanger=EXCHANGER_ADDRESS,
    xtz_amount=discounted_amount,
)


Properties
.key		tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc
.shell		['https://rpc.tzkt.io/ghostnet/']
.block_id	head

Payload
{'branch': 'BMTeSeYmqVRBxBHkJ4QrpTPrBY73YBGL8KZnmL6m821eVgrSytY',
 'contents': [{'amount': '400',
               'counter': '27211180',
               'destination': 'KT1Log35YrDTHBski4ZH7tJnzHedz5B2crb5',
               'fee': '1523',
               'gas_limit': '9804',
               'kind': 'transaction',
               'parameters': {'entrypoint': 'purchase_withdrawal_proxy',
                              'value': {'args': [{'args': [{'int': '1811'},
                                                           {'int': '414'},
                                                           {'string': '2025-03-16T12:58:56Z'},
                                                           {'string': 'tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc'},
                                                           {'bytes': '05009006'},
                                                         

- Transaction: https://ghostnet.tzkt.io/onebYiqk4fJW4Cx6JQVPiwUNcURg5r8iPja3rociNQbqiWbHioB/27211180

## Trying to purchase twice:
- Forcing the transaction to be executed

In [34]:
opg = (
    purchase_withdrawal_proxy.contract.purchase_withdrawal_proxy(
        withdrawal.as_tuple(),
        get_address(tezos_account),
        get_address(fast_withdrawal),
        EXCHANGER_ADDRESS,
    )
    .with_amount(discounted_amount)
    .as_transaction()
    .fill()
    .sign()
    .inject()
)
opg['hash']

'oocMYdrURyQ9nqnE53qCy8eJenbKzWF1KzG5Rn4KU5y68Twc43f'

## Making Additional Withdrawals:
### One with an incorrect payload that would be impossible to execute

In [42]:
from scripts.etherlink.xtz_fast_withdraw import (
    load_withdraw_precompile,
    make_fast_withdrawal,
)

precompile_contract = load_withdraw_precompile(
    XTZ_WITHDRAWAL_PRECOMPILE,
    web3
)
receipt = make_fast_withdrawal(
    etherlink_account=etherlink_account,
    web3=web3,
    precompile_contract=precompile_contract,
    target=get_address(tezos_account),
    fast_withdrawals_contract=fast_withdrawal.address,
    payload=bytes.fromhex('ff4f'),
    wei_amount=133 * 10**12,
)

click.echo(
    'Successfully initiated wrong XTZ Fast Withdrawal, tx hash: '
    + wrap(accent(receipt.transactionHash.hex()))
)

Successfully initiated wrong XTZ Fast Withdrawal, tx hash: `0xed920bc33fbbf47ee93cde3acd2a98ff84b698e58ba1e3638a59620622ef69ef`


### ~~Three~~ Four more with different base withdrawers and amounts

In [55]:
from scripts.etherlink import xtz_fast_withdraw
from random import randint

xtz_fast_withdraw.callback(
    target=get_address(tezos_account),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=randint(100, 1000) * 10**12,
    discounted_amount=randint(100, 1000),
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Fast Withdrawal contract: `KT18zhkJ9AohD17HaPuEiMfvjQC9horkn6Xb`
      * Payload bytes: `05009a0f`
      * Amount (mutez): `510`
      * Discounted amt (mutez): `986`
      * Fee (mutez): `-476`
Successfully initiated FA withdrawal, tx hash: `0x285d6d9ee9dcf8286618d3956b00d712c9c3982413f1f827f2b18622eb630314`


'0x285d6d9ee9dcf8286618d3956b00d712c9c3982413f1f827f2b18622eb630314'

- 1813: [L2](https://testnet.explorer.etherlink.com/tx/0x285d6d9ee9dcf8286618d3956b00d712c9c3982413f1f827f2b18622eb630314?tab=logs), [outbox](https://ghostnet-smart.tzkt.io/global/block/11221648/outbox/11221648/messages), not executed


In [56]:
xtz_fast_withdraw.callback(
    target=tezos_account.key.generate(export=False).public_key_hash(),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=randint(100, 1000) * 10**12,
    discounted_amount=randint(100, 1000),
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1KecET9vQXX4rBR7hNtHzCKx9JKHL1RBN4`
      * Fast Withdrawal contract: `KT18zhkJ9AohD17HaPuEiMfvjQC9horkn6Xb`
      * Payload bytes: `05008c08`
      * Amount (mutez): `176`
      * Discounted amt (mutez): `524`
      * Fee (mutez): `-348`
Successfully initiated FA withdrawal, tx hash: `0x6312b0fdf6df9af6c429230cb4ec281478f8c3de6ea62f02a42da0bdf9442433`


'0x6312b0fdf6df9af6c429230cb4ec281478f8c3de6ea62f02a42da0bdf9442433'

- 1814: [L2](https://testnet.explorer.etherlink.com/tx/0x6312b0fdf6df9af6c429230cb4ec281478f8c3de6ea62f02a42da0bdf9442433), [outbox](https://ghostnet-smart.tzkt.io/global/block/11221657/outbox/11221657/messages), not executed

In [57]:
xtz_fast_withdraw.callback(
    target=tezos_account.key.generate(export=False).public_key_hash(),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=randint(100, 1000) * 10**12,
    discounted_amount=randint(100, 1000),
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1WcH6fzgWQyCDFtuFcjxZLybRoBWRVtCXz`
      * Fast Withdrawal contract: `KT18zhkJ9AohD17HaPuEiMfvjQC9horkn6Xb`
      * Payload bytes: `0500a40e`
      * Amount (mutez): `462`
      * Discounted amt (mutez): `932`
      * Fee (mutez): `-470`
Successfully initiated FA withdrawal, tx hash: `0x65d7004a8a49c6805a2c0f529ecc161f8fadec762efff779ef7d9966cfb16e69`


'0x65d7004a8a49c6805a2c0f529ecc161f8fadec762efff779ef7d9966cfb16e69'

- 1815: [L2](https://testnet.explorer.etherlink.com/tx/0x65d7004a8a49c6805a2c0f529ecc161f8fadec762efff779ef7d9966cfb16e69), [outbox](https://ghostnet-smart.tzkt.io/global/block/11221659/outbox/11221659/messages), not executed

In [58]:
xtz_fast_withdraw.callback(
    target=tezos_account.key.generate(export=False).public_key_hash(),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=randint(100, 1000) * 10**12,
    discounted_amount=randint(100, 1000),
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1SYNmMi6CQ6r2PiR5wz8gkuHvANMTxW27S`
      * Fast Withdrawal contract: `KT18zhkJ9AohD17HaPuEiMfvjQC9horkn6Xb`
      * Payload bytes: `0500b701`
      * Amount (mutez): `105`
      * Discounted amt (mutez): `119`
      * Fee (mutez): `-14`
Successfully initiated FA withdrawal, tx hash: `0xa866f4ca3d1caddf7b8685138255833201aece0b5211455d37f22bb3a6e6fa51`


'0xa866f4ca3d1caddf7b8685138255833201aece0b5211455d37f22bb3a6e6fa51'

- 1816: [L2](https://testnet.explorer.etherlink.com/tx/0xa866f4ca3d1caddf7b8685138255833201aece0b5211455d37f22bb3a6e6fa51), [outbox](https://ghostnet-smart.tzkt.io/global/block/11221660/outbox/11221660/messages), not executed